# Option C: AI Agent Swarm

**Module:** 4.3 - Capstone Project (Domain 4: Production AI)
**Time:** 35-45 hours total
**Difficulty:** ⭐⭐⭐⭐⭐

---

## 🎯 Project Overview

Build a multi-agent system for complex tasks:
- **4+ Specialized Agents** - Each with unique capabilities
- **Agent Coordination** - Task planning and delegation
- **Tool Integration** - Code execution, search, APIs
- **Shared Memory** - Persistent context across agents
- **Human-in-the-Loop** - Safety and approval mechanisms

---

## 🎯 Learning Objectives

By completing this project, you will:
- [ ] Design specialized agents with distinct roles
- [ ] Implement agent communication protocols
- [ ] Build task decomposition and planning
- [ ] Create robust error handling and recovery
- [ ] Add human-in-the-loop safety controls

---

## 🌍 Real-World Context

Multi-agent systems are the frontier of AI:

| System | Agents | Capability |
|--------|--------|------------|
| **AutoGPT** | Planner + Executor | Autonomous task completion |
| **GPT-Engineer** | Designer + Coder + Tester | Full software development |
| **ChatDev** | CEO + CTO + Programmer + Tester | Simulated software company |
| **MetaGPT** | Multiple roles | Complex project generation |

With DGX Spark's 128GB memory, you can run multiple specialized agents simultaneously!

---

## 🧒 ELI5: What is an Agent Swarm?

> **Imagine you're building a house.** You don't hire one person to do everything:
>
> - **Architect** draws the plans
> - **Electrician** handles wiring
> - **Plumber** does the pipes
> - **Carpenter** builds the frame
> - **Project Manager** coordinates everyone
>
> **An agent swarm works the same way!** Each AI agent specializes:
> - **Researcher** finds information
> - **Coder** writes programs
> - **Analyst** examines data
> - **Coordinator** manages the team
>
> They communicate, share notes, and work together to solve problems too complex for any single agent.

---

## 🏗️ System Architecture

```
┌─────────────────────────────────────────────────────────────────────┐
│                         AI Agent Swarm                               │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│                    ┌──────────────────────┐                         │
│                    │     Coordinator      │                         │
│                    │   (Task Planning)    │                         │
│                    └──────────┬───────────┘                         │
│                               │                                      │
│         ┌─────────────────────┼─────────────────────┐               │
│         │                     │                     │               │
│         ▼                     ▼                     ▼               │
│  ┌──────────────┐    ┌──────────────┐    ┌──────────────┐          │
│  │  Researcher  │    │    Coder     │    │   Analyst    │          │
│  │    Agent     │    │    Agent     │    │    Agent     │          │
│  └──────┬───────┘    └──────┬───────┘    └──────┬───────┘          │
│         │                   │                   │                   │
│         ▼                   ▼                   ▼                   │
│  ┌──────────────┐    ┌──────────────┐    ┌──────────────┐          │
│  │ Web Search   │    │ Code Exec    │    │ Data Tools   │          │
│  │ RAG Lookup   │    │ File I/O     │    │ Visualization│          │
│  └──────────────┘    └──────────────┘    └──────────────┘          │
│                                                                      │
│  ┌────────────────────────────────────────────────────────────┐    │
│  │                    Shared Memory Store                      │    │
│  └────────────────────────────────────────────────────────────┘    │
│                                                                      │
│  ┌────────────────────────────────────────────────────────────┐    │
│  │              Human-in-the-Loop Safety Layer                 │    │
│  └────────────────────────────────────────────────────────────┘    │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

## Part 1: Agent Framework

In [ ]:
# Core Agent Framework

import torch
from datetime import datetime
from typing import List, Dict, Any, Optional, Callable
from dataclasses import dataclass, field
from enum import Enum
from abc import ABC, abstractmethod
import json
import uuid

print("🚀 OPTION C: AI AGENT SWARM")
print("="*60)
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print(f"\nGPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'Not available'}")

class MessageType(Enum):
    """Types of messages between agents."""
    TASK = "task"
    RESULT = "result"
    QUERY = "query"
    UPDATE = "update"
    ERROR = "error"

@dataclass
class AgentMessage:
    """Message passed between agents."""
    id: str
    sender: str
    recipient: str
    msg_type: MessageType
    content: Dict[str, Any]
    timestamp: datetime = field(default_factory=datetime.now)
    
    def to_dict(self) -> Dict:
        return {
            "id": self.id,
            "sender": self.sender,
            "recipient": self.recipient,
            "type": self.msg_type.value,
            "content": self.content,
            "timestamp": self.timestamp.isoformat(),
        }

@dataclass
class Tool:
    """A tool available to agents."""
    name: str
    description: str
    function: Callable
    parameters: Dict[str, Any]  # JSON Schema
    requires_approval: bool = False

class BaseAgent(ABC):
    """
    Base class for all agents in the swarm.
    """
    
    def __init__(
        self,
        name: str,
        role: str,
        model_name: str = "meta-llama/Llama-3.3-8B-Instruct",
        tools: List[Tool] = None,
    ):
        self.name = name
        self.role = role
        self.model_name = model_name
        self.tools = tools or []
        self.memory: List[Dict] = []  # Agent's local memory
        self._model = None
        self._tokenizer = None
    
    @property
    def system_prompt(self) -> str:
        """Get the agent's system prompt."""
        tools_desc = "\n".join([f"- {t.name}: {t.description}" for t in self.tools])
        return f"""You are {self.name}, a specialized AI agent.

Role: {self.role}

Available Tools:
{tools_desc if tools_desc else 'No tools available'}

When you need to use a tool, respond with:
TOOL: <tool_name>
ARGS: <json_arguments>

When you have completed a task, respond with:
RESULT: <your_result>

Always explain your reasoning before taking actions."""
    
    def _load_model(self):
        """Lazy load the model."""
        if self._model is not None:
            return
        
        from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
        
        print(f"Loading model for {self.name}: {self.model_name}")
        
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_quant_type="nf4",
        )
        
        self._model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            quantization_config=bnb_config,
            device_map="auto",
            trust_remote_code=True,
        )
        self._tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        print(f"✅ {self.name} model loaded")
    
    def think(self, task: str, context: str = "") -> str:
        """Generate a response for the given task."""
        self._load_model()
        
        messages = [
            {"role": "system", "content": self.system_prompt},
        ]
        
        # Add memory context
        if self.memory:
            memory_str = "\n".join([f"- {m['content'][:100]}..." for m in self.memory[-5:]])
            messages.append({"role": "system", "content": f"Recent memory:\n{memory_str}"})
        
        if context:
            messages.append({"role": "system", "content": f"Context:\n{context}"})
        
        messages.append({"role": "user", "content": task})
        
        prompt = self._tokenizer.apply_chat_template(messages, tokenize=False)
        inputs = self._tokenizer(prompt, return_tensors="pt").to("cuda")
        
        with torch.no_grad():
            outputs = self._model.generate(
                **inputs,
                max_new_tokens=1024,
                temperature=0.7,
                do_sample=True,
            )
        
        response = self._tokenizer.decode(
            outputs[0][inputs["input_ids"].shape[1]:],
            skip_special_tokens=True
        )
        
        # Store in memory
        self.memory.append({"task": task, "response": response[:500], "timestamp": datetime.now()})
        
        return response
    
    @abstractmethod
    def process_message(self, message: AgentMessage) -> AgentMessage:
        """Process an incoming message and return response."""
        pass

print("\n✅ Base agent framework defined")

---

## Part 2: Specialized Agents

In [ ]:
# Specialized Agent Implementations

class ResearcherAgent(BaseAgent):
    """Agent specialized in research and information gathering."""
    
    def __init__(self, tools: List[Tool] = None):
        super().__init__(
            name="Researcher",
            role="""You are a research specialist. Your job is to:
1. Find relevant information for given queries
2. Synthesize information from multiple sources
3. Provide accurate citations and sources
4. Summarize findings clearly""",
            tools=tools,
        )
    
    def process_message(self, message: AgentMessage) -> AgentMessage:
        if message.msg_type == MessageType.TASK:
            query = message.content.get("query", "")
            response = self.think(f"Research the following: {query}")
            
            return AgentMessage(
                id=str(uuid.uuid4()),
                sender=self.name,
                recipient=message.sender,
                msg_type=MessageType.RESULT,
                content={"research": response, "query": query}
            )
        return None

class CoderAgent(BaseAgent):
    """Agent specialized in writing and reviewing code."""
    
    def __init__(self, tools: List[Tool] = None):
        super().__init__(
            name="Coder",
            role="""You are a software engineer. Your job is to:
1. Write clean, efficient code
2. Debug and fix issues
3. Review code for problems
4. Explain code clearly

Always include comments and follow best practices.""",
            tools=tools,
        )
    
    def process_message(self, message: AgentMessage) -> AgentMessage:
        if message.msg_type == MessageType.TASK:
            task = message.content.get("task", "")
            language = message.content.get("language", "python")
            
            response = self.think(f"Write {language} code to: {task}")
            
            return AgentMessage(
                id=str(uuid.uuid4()),
                sender=self.name,
                recipient=message.sender,
                msg_type=MessageType.RESULT,
                content={"code": response, "task": task}
            )
        return None

class AnalystAgent(BaseAgent):
    """Agent specialized in data analysis."""
    
    def __init__(self, tools: List[Tool] = None):
        super().__init__(
            name="Analyst",
            role="""You are a data analyst. Your job is to:
1. Analyze data and find patterns
2. Create visualizations
3. Generate insights and recommendations
4. Validate hypotheses with data

Always support conclusions with evidence.""",
            tools=tools,
        )
    
    def process_message(self, message: AgentMessage) -> AgentMessage:
        if message.msg_type == MessageType.TASK:
            analysis_task = message.content.get("task", "")
            data = message.content.get("data", "")
            
            response = self.think(
                f"Analyze the following:\n{analysis_task}",
                context=f"Data:\n{data}" if data else ""
            )
            
            return AgentMessage(
                id=str(uuid.uuid4()),
                sender=self.name,
                recipient=message.sender,
                msg_type=MessageType.RESULT,
                content={"analysis": response, "task": analysis_task}
            )
        return None

class CoordinatorAgent(BaseAgent):
    """Agent that coordinates other agents."""
    
    def __init__(self, agent_pool: Dict[str, BaseAgent] = None):
        super().__init__(
            name="Coordinator",
            role="""You are the team coordinator. Your job is to:
1. Break down complex tasks into subtasks
2. Assign tasks to the right specialist
3. Monitor progress and handle issues
4. Synthesize results into final output

Available specialists: Researcher, Coder, Analyst""",
        )
        self.agent_pool = agent_pool or {}
    
    def plan_task(self, task: str) -> List[Dict]:
        """Decompose a task into subtasks for specialists."""
        planning_prompt = f"""Break down this task into subtasks:

Task: {task}

Available agents:
- Researcher: For information gathering
- Coder: For writing code
- Analyst: For data analysis

Output a JSON list of subtasks with format:
[{{"agent": "AgentName", "task": "description", "depends_on": []}}]

Only output the JSON, nothing else."""
        
        response = self.think(planning_prompt)
        
        # Parse plan
        try:
            import re
            json_match = re.search(r'\[.*\]', response, re.DOTALL)
            if json_match:
                return json.loads(json_match.group())
        except:
            pass
        
        return [{"agent": "Coordinator", "task": task, "depends_on": []}]
    
    def process_message(self, message: AgentMessage) -> AgentMessage:
        if message.msg_type == MessageType.TASK:
            task = message.content.get("task", "")
            
            # Plan the task
            plan = self.plan_task(task)
            
            # Execute plan
            results = []
            for subtask in plan:
                agent_name = subtask.get("agent")
                if agent_name in self.agent_pool:
                    agent = self.agent_pool[agent_name]
                    sub_msg = AgentMessage(
                        id=str(uuid.uuid4()),
                        sender=self.name,
                        recipient=agent_name,
                        msg_type=MessageType.TASK,
                        content={"task": subtask.get("task", ""), "query": subtask.get("task", "")}
                    )
                    result = agent.process_message(sub_msg)
                    if result:
                        results.append({"agent": agent_name, "result": result.content})
            
            # Synthesize results
            synthesis_prompt = f"""Synthesize these results into a final answer:

Original task: {task}

Results from team:
{json.dumps(results, indent=2)}

Provide a comprehensive final answer."""
            
            final_answer = self.think(synthesis_prompt)
            
            return AgentMessage(
                id=str(uuid.uuid4()),
                sender=self.name,
                recipient=message.sender,
                msg_type=MessageType.RESULT,
                content={
                    "final_answer": final_answer,
                    "plan": plan,
                    "intermediate_results": results,
                }
            )
        return None

print("✅ Specialized agents defined")
print("\nAvailable agents:")
print("  • ResearcherAgent - Information gathering")
print("  • CoderAgent - Code writing")
print("  • AnalystAgent - Data analysis")
print("  • CoordinatorAgent - Task orchestration")

---

## Part 3: Shared Memory & Communication

In [ ]:
# Shared Memory System

class SharedMemory:
    """
    Shared memory store for agent swarm.
    
    Provides:
    - Key-value storage
    - Message history
    - Artifact storage (code, data, etc.)
    """
    
    def __init__(self):
        self.store: Dict[str, Any] = {}
        self.messages: List[AgentMessage] = []
        self.artifacts: Dict[str, Dict] = {}
    
    def set(self, key: str, value: Any, agent: str = None):
        """Store a value."""
        self.store[key] = {
            "value": value,
            "agent": agent,
            "timestamp": datetime.now(),
        }
    
    def get(self, key: str) -> Any:
        """Retrieve a value."""
        entry = self.store.get(key)
        return entry["value"] if entry else None
    
    def log_message(self, message: AgentMessage):
        """Log a message to history."""
        self.messages.append(message)
    
    def get_conversation_history(
        self,
        agent: str = None,
        limit: int = 10
    ) -> List[AgentMessage]:
        """Get recent message history."""
        if agent:
            relevant = [m for m in self.messages if m.sender == agent or m.recipient == agent]
        else:
            relevant = self.messages
        return relevant[-limit:]
    
    def save_artifact(
        self,
        name: str,
        content: str,
        artifact_type: str,
        agent: str
    ):
        """Save an artifact (code, document, etc.)."""
        self.artifacts[name] = {
            "content": content,
            "type": artifact_type,
            "agent": agent,
            "timestamp": datetime.now(),
        }
    
    def get_artifact(self, name: str) -> Dict:
        """Retrieve an artifact."""
        return self.artifacts.get(name)
    
    def summary(self) -> str:
        """Get a summary of memory contents."""
        return f"""Shared Memory Summary:
- Stored values: {len(self.store)}
- Messages: {len(self.messages)}
- Artifacts: {len(self.artifacts)}
- Keys: {list(self.store.keys())[:10]}"""

print("✅ Shared memory system defined")

---

## Part 4: Human-in-the-Loop Safety

In [ ]:
# Safety & Approval System

class SafetyLevel(Enum):
    LOW = "low"        # Auto-approve
    MEDIUM = "medium"  # Log and notify
    HIGH = "high"      # Require approval
    CRITICAL = "critical"  # Block and alert

@dataclass
class ActionRequest:
    """Request for an action that may need approval."""
    id: str
    agent: str
    action: str
    parameters: Dict[str, Any]
    safety_level: SafetyLevel
    justification: str
    approved: Optional[bool] = None
    approval_notes: str = ""

class HumanInTheLoop:
    """
    Human-in-the-loop safety system.
    
    Controls:
    - Action approval
    - Safety monitoring
    - Emergency stop
    """
    
    def __init__(self, auto_approve_low: bool = True):
        self.auto_approve_low = auto_approve_low
        self.pending_approvals: Dict[str, ActionRequest] = {}
        self.action_log: List[ActionRequest] = []
        self.emergency_stop = False
        
        # Define risky actions
        self.risky_patterns = [
            ("rm -rf", SafetyLevel.CRITICAL),
            ("delete", SafetyLevel.HIGH),
            ("drop table", SafetyLevel.CRITICAL),
            ("sudo", SafetyLevel.HIGH),
            ("password", SafetyLevel.HIGH),
            ("api_key", SafetyLevel.HIGH),
        ]
    
    def assess_safety(self, action: str, params: Dict) -> SafetyLevel:
        """Assess the safety level of an action."""
        action_lower = action.lower()
        params_str = json.dumps(params).lower()
        
        for pattern, level in self.risky_patterns:
            if pattern in action_lower or pattern in params_str:
                return level
        
        return SafetyLevel.LOW
    
    def request_approval(
        self,
        agent: str,
        action: str,
        parameters: Dict[str, Any],
        justification: str = ""
    ) -> ActionRequest:
        """
        Request approval for an action.
        
        Returns ActionRequest with approval status.
        """
        if self.emergency_stop:
            return ActionRequest(
                id=str(uuid.uuid4()),
                agent=agent,
                action=action,
                parameters=parameters,
                safety_level=SafetyLevel.CRITICAL,
                justification=justification,
                approved=False,
                approval_notes="Emergency stop active"
            )
        
        safety_level = self.assess_safety(action, parameters)
        
        request = ActionRequest(
            id=str(uuid.uuid4()),
            agent=agent,
            action=action,
            parameters=parameters,
            safety_level=safety_level,
            justification=justification,
        )
        
        # Auto-approve low risk if enabled
        if safety_level == SafetyLevel.LOW and self.auto_approve_low:
            request.approved = True
            request.approval_notes = "Auto-approved (low risk)"
        elif safety_level == SafetyLevel.CRITICAL:
            request.approved = False
            request.approval_notes = "Blocked (critical risk)"
        else:
            # Add to pending for human review
            self.pending_approvals[request.id] = request
            print(f"\n⚠️ APPROVAL NEEDED for {agent}:")
            print(f"   Action: {action}")
            print(f"   Safety: {safety_level.value}")
            print(f"   ID: {request.id}")
        
        self.action_log.append(request)
        return request
    
    def approve(self, request_id: str, notes: str = "") -> bool:
        """Approve a pending request."""
        if request_id in self.pending_approvals:
            request = self.pending_approvals[request_id]
            request.approved = True
            request.approval_notes = notes
            del self.pending_approvals[request_id]
            return True
        return False
    
    def deny(self, request_id: str, notes: str = "") -> bool:
        """Deny a pending request."""
        if request_id in self.pending_approvals:
            request = self.pending_approvals[request_id]
            request.approved = False
            request.approval_notes = notes
            del self.pending_approvals[request_id]
            return True
        return False
    
    def emergency_stop_toggle(self, stop: bool = True):
        """Toggle emergency stop."""
        self.emergency_stop = stop
        if stop:
            print("🚨 EMERGENCY STOP ACTIVATED")
        else:
            print("✅ Emergency stop deactivated")

print("✅ Human-in-the-loop safety system defined")

---

## Part 5: Agent Swarm Orchestration

In [ ]:
# Complete Agent Swarm

class AgentSwarm:
    """
    Complete agent swarm system.
    
    Combines:
    - Specialized agents
    - Shared memory
    - Human-in-the-loop safety
    - Task orchestration
    """
    
    def __init__(self):
        # Initialize components
        self.memory = SharedMemory()
        self.safety = HumanInTheLoop()
        
        # Create agents
        self.agents: Dict[str, BaseAgent] = {
            "Researcher": ResearcherAgent(),
            "Coder": CoderAgent(),
            "Analyst": AnalystAgent(),
        }
        
        # Create coordinator with access to agent pool
        self.coordinator = CoordinatorAgent(agent_pool=self.agents)
        self.agents["Coordinator"] = self.coordinator
    
    def execute_task(
        self,
        task: str,
        require_approval: bool = False
    ) -> Dict[str, Any]:
        """
        Execute a complex task using the agent swarm.
        
        Args:
            task: Task description
            require_approval: Whether to require human approval
            
        Returns:
            Execution results
        """
        print(f"\n🚀 Starting task: {task[:100]}...")
        print("="*60)
        
        # Check safety
        if require_approval:
            approval = self.safety.request_approval(
                agent="Coordinator",
                action="execute_task",
                parameters={"task": task},
                justification="User-requested approval"
            )
            if not approval.approved:
                return {"status": "blocked", "reason": approval.approval_notes}
        
        # Create initial message
        initial_message = AgentMessage(
            id=str(uuid.uuid4()),
            sender="User",
            recipient="Coordinator",
            msg_type=MessageType.TASK,
            content={"task": task}
        )
        
        # Log message
        self.memory.log_message(initial_message)
        
        # Execute via coordinator
        result = self.coordinator.process_message(initial_message)
        
        # Log result
        if result:
            self.memory.log_message(result)
            self.memory.set("last_result", result.content, "Coordinator")
        
        print("\n" + "="*60)
        print("✅ Task complete")
        
        return result.content if result else {"status": "error", "reason": "No result"}
    
    def chat(self, message: str) -> str:
        """Simple chat interface to the swarm."""
        result = self.execute_task(message)
        return result.get("final_answer", str(result))
    
    def status(self) -> str:
        """Get swarm status."""
        return f"""Agent Swarm Status:
- Agents: {', '.join(self.agents.keys())}
- {self.memory.summary()}
- Safety: {'🚨 STOPPED' if self.safety.emergency_stop else '✅ Active'}
- Pending approvals: {len(self.safety.pending_approvals)}"""

print("✅ Agent Swarm ready")
print("\nUsage:")
print("  swarm = AgentSwarm()")
print("  result = swarm.execute_task('Research AI trends and write a summary')")

---

## 🎉 Checkpoint & Next Steps

You now have the foundation for an AI Agent Swarm:

- ✅ Base agent framework
- ✅ Specialized agents (Researcher, Coder, Analyst)
- ✅ Coordinator for task planning
- ✅ Shared memory system
- ✅ Human-in-the-loop safety

### Implementation Roadmap

**Week 1-2:** Core agent development
- [ ] Add more specialized agents
- [ ] Implement robust tool execution
- [ ] Add web search tools
- [ ] Add code execution sandbox

**Week 3:** Communication & memory
- [ ] Enhance shared memory with vector search
- [ ] Implement agent-to-agent communication
- [ ] Add conversation persistence

**Week 4:** Safety & robustness
- [ ] Comprehensive safety rules
- [ ] Error recovery mechanisms
- [ ] Timeout handling

**Week 5-6:** Evaluation & documentation
- [ ] Create task benchmarks
- [ ] Measure success rates
- [ ] Document the system

---

In [ ]:
# Cleanup
import gc

if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

print("✅ Ready to build your Agent Swarm!")